In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.mixed_precision import set_global_policy
import re
import gc

set_global_policy('mixed_float16')

Объявляем константы

In [28]:
PREDICT_SEQUENCE_LENGTH = 5 # Количество предыдущих слов для предсказания следующего
EMBEDDING_DIM = 32
LSTM_UNITS = 100
EPOCHS = 30
BATCH_SIZE = 32
MAX_WORDS = 20000
SEED_WORDS = "гарри поттер открыл дверь и"
WORD_COUNT_TO_GENERATE = 50

Подготовка данных

In [5]:
raw_text_data = ""
with open('hpmor_ru.txt', 'r', encoding='utf-8') as f:
  raw_text_data = f.read()

def clean_text(text):
  text = text.lower()
  text = re.sub(r'\s+', ' ', text).strip()
  return text

text_data = clean_text(raw_text_data)
corpus = text_data.split('. ')
print(text_data[:50])

глава 1. крайне маловероятный день глава 2. всё, в


Токенизация на уровне слов

In [16]:
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts([text_data])

total_words = len(tokenizer.word_index) + 1
print(f"размер словаря: {total_words}")

input_sequences = []
all_words = tokenizer.texts_to_sequences([text_data])[0]

for i in range(PREDICT_SEQUENCE_LENGTH, len(all_words)):
    seq = all_words[i - PREDICT_SEQUENCE_LENGTH : i + 1]
    input_sequences.append(seq)

max_sequence_len_words = PREDICT_SEQUENCE_LENGTH + 1
sequences = np.array(input_sequences)

X = sequences[:MAX_WORDS,:-1]
y = sequences[:MAX_WORDS,-1]

print(f"Количество обучающих последовательностей: {len(X)}")
print(f"Длина входной последовательности для модели (X[0]): {len(X[0])} слов")

размер словаря: 51913
Количество обучающих последовательностей: 20000
Длина входной последовательности для модели (X[0]): 5 слов


Создание модели (Двунаправленная LSTM)

In [17]:
model = Sequential([
    Embedding(input_dim=total_words, output_dim=EMBEDDING_DIM, input_length=PREDICT_SEQUENCE_LENGTH),
    Bidirectional(LSTM(LSTM_UNITS, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(LSTM_UNITS)),
    Dropout(0.2),
    Dense(EMBEDDING_DIM, activation='relu'),
    Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Обучение модели

In [23]:
class TextGenerator(Sequence):
    def __init__(self, text, tokenizer, seq_length, batch_size):
        self.text = text
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.indices = np.arange(len(text) - seq_length)

    def __len__(self):
        return (len(self.indices) // self.batch_size)

    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size : (idx+1)*self.batch_size]
        X = np.zeros((self.batch_size, self.seq_length))
        y = np.zeros((self.batch_size))

        for i, start_idx in enumerate(batch_indices):
            seq = self.text[start_idx : start_idx + self.seq_length + 1]
            X[i] = seq[:-1]
            y[i] = seq[-1]

        return X, y

class MemoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()

train_generator = TextGenerator(all_words[:MAX_WORDS], tokenizer, PREDICT_SEQUENCE_LENGTH, batch_size=8)
history = model.fit(train_generator, epochs=EPOCHS, batch_size=128, verbose=1, callbacks=[MemoryCallback()])
# history = model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 50s 20ms/step - accuracy: 0.0551 - loss: 7.2253
Epoch 2/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 45s 18ms/step - accuracy: 0.0639 - loss: 6.9161
Epoch 3/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - accuracy: 0.0741 - loss: 6.7641
Epoch 4/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - accuracy: 0.0757 - loss: 6.5358
Epoch 5/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - accuracy: 0.0833 - loss: 6.2604
Epoch 6/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - accuracy: 0.0867 - loss: 6.0777
Epoch 7/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 83s 17ms/step - accuracy: 0.0892 - loss: 5.9025
Epoch 8/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - accuracy: 0.0916 - loss: 5.7275
Epoch 9/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 83s 17ms/step - accuracy: 0.1040 - loss: 5.5195
Epoch 10/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - accuracy: 0.1155 - loss: 5.3151
Epoch 11/30
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 44s 17ms/step - accuracy: 0.1167 - loss: 5.1573
Epoch 12

Оцениваем качество

In [24]:
final_loss = history.history['loss'][-1]
final_accuracy = history.history['accuracy'][-1]
print(f"\nФинальное значение функции потерь (categorical cross-entropy): {final_loss}")
print(f"Финальная точность (accuracy): {final_accuracy}")
perplexity = np.exp(final_loss)
print(f"Примерная перплексия: {perplexity}")


Финальное значение функции потерь (categorical cross-entropy): 3.1098713874816895
Финальная точность (accuracy): 0.3065226078033447
Примерная перплексия: 22.418160959190125


Функция для генерации текста

In [29]:
def generate_text_words(model, tokenizer, seed_text, num_words_to_generate, sequence_length, temperature=1.0):
  generated_text = seed_text.lower()
  current_words = tokenizer.texts_to_sequences([seed_text.lower()])[0]

  if len(current_words) < sequence_length:
    print(f"Предупреждение: Затравочный текст '{seed_text}' короче sequence_length ({sequence_length}). Результат может быть неоптимальным.")

  result_words = list(current_words)

  for _ in range(num_words_to_generate):
    padded_sequence = pad_sequences([current_words], maxlen=sequence_length, padding='pre', truncating='pre')

    if padded_sequence.shape[1] == 0:
      print("Ошибка: Последовательность для предсказания пуста.")
      break

    y_pred_proba = model.predict(padded_sequence, verbose=0)[0]

    y_pred_proba = np.asarray(y_pred_proba).astype('float64')
    y_pred_proba = np.log(y_pred_proba + 1e-9) / temperature
    exp_preds = np.exp(y_pred_proba)
    preds = exp_preds / np.sum(exp_preds)

    if np.isnan(preds).all() or np.isinf(preds).all():
      print("Предупреждение: NaN/inf в вероятностях, выбираем случайное слово.")
      next_word_index = np.random.choice(len(preds))
    elif np.isclose(np.sum(preds), 1.0):
      next_word_index = np.random.choice(len(preds), p=preds)
    else:
      print(f"Предупреждение: сумма вероятностей {np.sum(preds)} не равна 1, используется argmax.")
      next_word_index = np.argmax(preds)

    if next_word_index == 0:
      print("Предупреждение: Попытка сгенерировать индекс 0. Пропускаем.")
      continue


    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == next_word_index:
        output_word = word
        break

    if output_word:
      generated_text += " " + output_word
      current_words.append(next_word_index)
      if len(current_words) > sequence_length:
        current_words = current_words[1:]
    else:
      print(f"Предупреждение: не найдено слово для индекса {next_word_index}")


  return generated_text

Теперь сгенерируем текст

In [30]:
print(f"Сид: {SEED_WORDS}")

for temp in [0.5, 0.8, 1.0, 1.2]:
  generated_output = generate_text_words(
    model, tokenizer, SEED_WORDS,
    num_words_to_generate=WORD_COUNT_TO_GENERATE,
    sequence_length=PREDICT_SEQUENCE_LENGTH,
    temperature=temp
  )

  print(f"\nТемпература: {temp}")
  print(generated_output)
  print("--------------------")

Сид: гарри поттер открыл дверь и

Температура: 0.5
гарри поттер открыл дверь и <unk> может из прав и того потому что он захлопнул способ разговаривал — вероятно бы » цвет не чистого <unk> <unk> его джеймс гарри вздохнула эванс работе понизил стороне <unk> питья положила ведьм расслабилась как ему неизвестно <unk> делу с вами <unk> — профессор макгонагалл — сказал гарри — просто
--------------------

Температура: 0.8
гарри поттер открыл дверь и гарри день но же мои не должны ничего посмотрел — гм — ты он широко раскрыл плата — переспросила — я говорите — это больше в которую мистер поттер — сказала макгонагалл — она так не нравится это мешочек — кивнула — гм мистер поттер часть этих хотел и прикоснуться
--------------------

Температура: 1.0
гарри поттер открыл дверь и потому что у него будет суммы слов помощниц вещи у например всегда хотел <unk> к раз множество идея вселенной его головой — тому ахава — полагаю же теперь вам ему когда вы живы появилось «золото» общий достал <unk> фундам